In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
 
 
def fetch_results(search_term, number_results):
    language_code ='en'
    assert isinstance(search_term, str), 'Search term must be a string'
    assert isinstance(number_results, int), 'Number of results must be an integer'
    escaped_search_term = search_term.replace(' ', '+')
 
    google_url = 'https://www.google.com/search?q={}&num={}&hl={}'.format(escaped_search_term, number_results, language_code)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()
 
    return search_term, response.text

def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')
 
    found_results = []
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    for result in result_block:
 
        link = result.find('a', href=True)
        title = result.find('h3')
        description = result.find('span', attrs={'class': 'st'})
        if link and title:
            link = link['href']
            title = title.get_text()
           
            if description:
                description = description.get_text()
            if link != '#':
                found_results.append({'keyword': keyword, 'rank': rank, 'title': title, 'description': description, 
                                      'link': link})
                rank += 1
    return found_results

def scrape_google(search_term, number_results):
    language_code = 'en'
    try:
        keyword, html = fetch_results(search_term, number_results)
        results = parse_results(html, keyword)
        return pd.DataFrame(results) 
    except AssertionError:
        raise Exception("Incorrect arguments parsed to function")
    except requests.HTTPError:
        raise Exception("You appear to have been blocked by Google")
    except requests.RequestException:
        raise Exception("Appears to be an issue with your connection")
 


In [33]:
scrape_google("boston roofing",11)

,keyword,rank,title,description,link
0,boston roofing,1,"BOSTON ROOFING, Commercial roofers in Greater ...",None,http://bostonroofing.com/
1,boston roofing,2,"Greater Boston Roofing • Top Quality Roofing, ...",None,https://greaterbostonroofing.com/
2,boston roofing,3,BOSTON ROOFING - Home | Facebook,None,https://www.facebook.com/BostonRoofing/
3,boston roofing,4,Boston Roofing and Gutters,None,https://www.bostonroofingandgutters.com/
4,boston roofing,5,Boston Roofing & Aluminum,None,https://www.bostonroofing.ca/
5,boston roofing,6,Boston Roofing | LinkedIn,None,https://www.linkedin.com/company/boston-roofing
6,boston roofing,7,Top 10 Best Roofing Contractors in Boston MA -...,None,https://www.angi.com/companylist/boston/roofin...
7,boston roofing,8,"THE BEST 10 Roofing in Boston, MA - Last Updat...",None,https://www.yelp.com/search?cflt=roofing&find_...
8,boston roofing,9,Boston Roofing And Gutters 277 Newport Ave Qui...,None,https://www.mapquest.com/us/massachusetts/bost...
9,boston roofing,10,1 Roofing Contractors | The Boston Roofers,None,https://roofingbostonma.com/
